# 合并ksrpc+frpc两阻塞代码于单Notebook

In [ ]:
def run_command(args):
    import select
    import subprocess
    from IPython.display import clear_output
    
    try:
        # 使用 subprocess 运行外部命令
        process = subprocess.Popen(
            args,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=True,
        )
        i = 0
        # 实时输出日志
        while process.poll() is None:
            # 检查 stdout/stderr 是否有数据
            rlist, _, _ = select.select([process.stdout, process.stderr], [], [], 0.1)
            for stream in rlist:
                line = stream.readline()
                if line:
                    i+=1
                    if i%100==0:
                        clear_output(wait=True)
                    print(line.strip())
                
        # 等待进程结束
        process.wait()
    except KeyboardInterrupt:
        print(args, "已中断")
    except Exception as e:
        print(args, "启动失败: ", e)
    finally:
        print(args, "已终止")

In [ ]:
import multiprocessing
import time
from contextlib import contextmanager

@contextmanager
def process_manager(*processes):
    try:
        for p in processes:
            p.start()
        yield
    except KeyboardInterrupt:
        print("\n收到终止信号，关闭子进程...")
        time.sleep(1)  # 给子进程时间处理
    finally:
        for p in processes:
            if p.is_alive():
                print(f"强制终止 {p.name}")
                p.terminate()
            p.join(timeout=1.0)
            if p.is_alive():
                print(f"警告: {p.name} 未能正常终止")

if __name__ == '__main__':
    # 创建进程时指定名称便于调试
    p1 = multiprocessing.Process(target=lambda :run_command(["python", "-u", "-m", "ksrpc.run_simple"]), name="ksrpc")
    p2 = multiprocessing.Process(target=lambda :run_command(["./frpc", "-c", "./frpc.toml"]), name="frpc")
    
    try:
        with process_manager(p1, p2):
            # 主线程只需等待，由process_manager处理退出
            while True:
                time.sleep(2)
    except Exception as e:
        print(f"主进程错误: {e}")
    finally:
        print("所有服务已关闭")